DATASETS ARMADOS POR SEPARADO, PARA ACELERAR CONSULTAS


ANÁLISIS DE SENTIMIENTOS + MACHINE LEARNING


importar librerias


In [2]:
import pandas as pd  # Pandas se utiliza para el manejo y análisis de datos tabulares
import pyarrow as pa  # PyArrow se utiliza para trabajar con formatos de datos columnares y eficientes como Parquet
import pyarrow.parquet as pq  # Importamos Parquet

import ast  # AST (Abstract Syntax Trees) se utiliza para interpretar expresiones Python
import gzip
import json  # JSON se utiliza para trabajar con datos en formato JSON
import os  # OS proporciona funciones para interactuar con el sistema oper
from typing import List, Tuple
# Importación del módulo gc para la recolección de basura
import gc

# Importación del módulo pickle para la serialización de objetos
import pickle

# Importación del módulo warnings para controlar advertencias
import warnings

# Importación de pandas y se asigna el alias 'pd' para su uso
import pandas as pd

Llamar a los datasets que creamos anteriormente

In [3]:
df_steam_games =   pd.read_parquet('Datasets\pdf_SteamGames.parquet')
df_user_reviews = pd.read_parquet(r'Datasets\new_user_reviews.parquet')
df_user_items = pd.read_parquet(r'Datasets\new_df_users_items.parquet')


Realizamos la primera consulta: "def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora"

In [4]:
def developer(desarrollador:str):
    # Filtrar por desarrollador
    df_dev = df_steam_games[df_steam_games['developer'] == desarrollador]
    
    # Obtener la cantidad de ítems por año
    items_por_anio = df_dev.groupby('release_date').size()
    
    # Obtener el porcentaje de contenido Free por año
    contenido_free_por_anio = df_dev[df_dev['price'] == 0].groupby('release_date').size()
    porcentaje_free_por_anio = (contenido_free_por_anio / items_por_anio * 100).round(2)
    
    # Reemplazar valores NaN en "Contenido Free" con "0%"
    porcentaje_free_por_anio.fillna(0, inplace=True)
    
    # Crear un DataFrame con los resultados
    result_df = pd.DataFrame({'Año': items_por_anio.index,
                              'Cantidad de Items': items_por_anio.values,
                              'Contenido Free': porcentaje_free_por_anio.values})
    
    # Ordenar el DataFrame por año
    result_df = result_df.sort_values(by='Año').reset_index(drop=True)
    
    # Formatear los valores en la columna "Contenido Free"
    result_df['Contenido Free'] = result_df['Contenido Free'].astype(str) + '%'
    
    # Imprimir el DataFrame formateado
    print(result_df.to_string(index=False))

# Ejemplo de uso de la función developer
developer('Valve')

 Año  Cantidad de Items Contenido Free
1998                  1           0.0%
1999                  1           0.0%
2000                  2           0.0%
2001                  1           0.0%
2003                  1           0.0%
2004                  5           0.0%
2006                  2           0.0%
2007                  3         33.33%
2008                  1           0.0%
2009                  1           0.0%
2010                  2          50.0%
2011                  1           0.0%
2012                  1           0.0%
2016                  1         100.0%
2017                  2           0.0%


Ahora, para la segunda consulta, debemos crear un nuevo dataframe que combine varias columnas clave para realizarla. "def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items."

In [5]:
# Fusionar los tres DataFrames basados en las columnas 'id', 'item_id' e 'item_id'
merged_df = pd.merge(df_steam_games, df_user_items, left_on=['id', 'app_name'], right_on=['item_id', 'item_name'])
merged_df = pd.merge(merged_df, df_user_reviews, left_on=['user_id', 'item_id'], right_on=['user_id', 'item_id'])

# Seleccionar solo las columnas necesarias para procesar el pedido
df_segunda_consulta = merged_df[['user_id','item_id', 'price', 'items_count', 'recommend', 'review']]

# Mostrar las primeras filas del nuevo DataFrame para verificar
print(df_segunda_consulta.head())

              user_id   item_id  price  items_count  recommend  \
0        InstigatorAU  282010.0   9.99          112       True   
1  EizanAratoFujimaki      70.0   9.99          391       True   
2            GamerFag      70.0   9.99           64       True   
3   76561198020928326      70.0   9.99          110       True   
4           Bluegills      70.0   9.99         1280       True   

                                              review  
0                       Oh, where do i start? DOS...  
1  ADVERTENCIA:Este Análisis contiene posibles Sp...  
2  If you own any half life but not this one that...  
3  this game has had me hooked for over 4 years a...  
4  This is, by far my most favourite game I have ...  


Con este nuevo dataframe, podremos realizar la consulta tranquilamente.

In [6]:
df_segunda_consulta.to_parquet('Datasets/df_segunda_consulta.parquet')

In [7]:
def userdata(User_id: str):
    # Filtrar el DataFrame para obtener las filas correspondientes al User_id dado
    user_data = df_segunda_consulta[df_segunda_consulta['user_id'] == User_id]
    
    # Calcular la cantidad total de dinero gastado por el usuario
    money_spent = user_data['price'].sum()
    
    # Calcular el porcentaje de recomendación en base a las revisiones
    recommend_percentage = (user_data['recommend'].mean()) * 100
    
    # Obtener la cantidad de items del usuario
    items_count = user_data['items_count'].sum()
    
    # Retornar un diccionario con los resultados
    return {
        "Usuario": User_id,
        "Dinero gastado": f"{money_spent} USD",
        "% de recomendación": f"{recommend_percentage}%",
        "Cantidad de items": items_count
    }

In [8]:
print(userdata("zerzang"))

{'Usuario': 'zerzang', 'Dinero gastado': '84.94 USD', '% de recomendación': '80.0%', 'Cantidad de items': 600}


Ahora, pasamos a la tercera consulta: "def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}"

In [9]:
tercer_merged_df = pd.merge(df_steam_games, df_user_items, left_on=['id', 'app_name'], right_on=['item_id', 'item_name'])
tercer_merged_df_final = pd.merge(tercer_merged_df, df_user_reviews, left_on=['user_id', 'item_id'], right_on=['user_id', 'item_id'])
# Seleccionar solo las columnas necesarias para procesar el pedido
df_tercera_consulta = tercer_merged_df_final[['user_id','item_id', 'genres', 'playtime_forever', 'year',]]

In [10]:
print(df_tercera_consulta.head())
df_tercera_consulta

              user_id   item_id  \
0        InstigatorAU  282010.0   
1  EizanAratoFujimaki      70.0   
2            GamerFag      70.0   
3   76561198020928326      70.0   
4           Bluegills      70.0   

                                              genres  playtime_forever  year  
0  ['Action', 'Indie', 'Racing'], ['Racing', 'Cla...               466  2014  
1  ['Action'], ['FPS', 'Classic', 'Sci-fi', 'Sing...              1395  2015  
2  ['Action'], ['FPS', 'Classic', 'Sci-fi', 'Sing...               590  2011  
3  ['Action'], ['FPS', 'Classic', 'Sci-fi', 'Sing...              5599  2014  
4  ['Action'], ['FPS', 'Classic', 'Sci-fi', 'Sing...                64  2013  


,user_id,item_id,genres,playtime_forever,year
0,InstigatorAU,282010.0,"['Action', 'Indie', 'Racing'], ['Racing', 'Cla...",466,2014
1,EizanAratoFujimaki,70.0,"['Action'], ['FPS', 'Classic', 'Sci-fi', 'Sing...",1395,2015
2,GamerFag,70.0,"['Action'], ['FPS', 'Classic', 'Sci-fi', 'Sing...",590,2011
3,76561198020928326,70.0,"['Action'], ['FPS', 'Classic', 'Sci-fi', 'Sing...",5599,2014
4,Bluegills,70.0,"['Action'], ['FPS', 'Classic', 'Sci-fi', 'Sing...",64,2013
...,...,...,...,...,...
31786,76561198015050660,80.0,"['Action'], 'FPS', 'Shooter', 'Multiplayer', '...",2416,2014
31787,76561198023508728,80.0,"['Action'], 'FPS', 'Shooter', 'Multiplayer', '...",15,2014
31788,green290,80.0,"['Action'], 'FPS', 'Shooter', 'Multiplayer', '...",442,2015
31789,174gamecuman700kngkakak,80.0,"['Action'], 'FPS', 'Shooter', 'Multiplayer', '...",72,2014


In [11]:
def UserForGenre(genero: str, df_tercera_consulta):
    # Filtrar el DataFrame por el género dado
    filtered_df = df_tercera_consulta[df_tercera_consulta['genres'].str.contains(genero, case=False)]
    print("Número de filas después de filtrar por género:", len(filtered_df))
    
    # Calcular la suma de horas jugadas por usuario para el género dado
    user_hours = filtered_df.groupby('user_id')['playtime_forever'].sum()
    
    # Encontrar el usuario con más horas jugadas para el género dado
    usuario_max_horas = user_hours.idxmax()
    print("Usuario con más horas jugadas para {}: {}".format(genero, usuario_max_horas))
    
    # Filtrar el DataFrame para obtener solo las filas del usuario con más horas jugadas para el género dado
    user_genre_df = filtered_df[(filtered_df['user_id'] == usuario_max_horas) & (filtered_df['genres'].str.contains(genero, case=False))]
    
    # Obtener las horas jugadas por año de lanzamiento para ese usuario y ese género
    horas_por_anio = user_genre_df.groupby('year')['playtime_forever'].sum().reset_index()
    print("Horas jugadas por año para {}: ".format(usuario_max_horas))
    print(horas_por_anio)
    
    horas_por_anio = [{"Año": row['year'], "Horas": row['playtime_forever']} for index, row in horas_por_anio.iterrows()]
    
    # Retornar el resultado
    return {"Usuario con más horas jugadas para {}: {}".format(genero, usuario_max_horas): usuario_max_horas,
            "Horas jugadas": horas_por_anio}



In [12]:
resultado = UserForGenre('Action', df_tercera_consulta)
print(resultado)

Número de filas después de filtrar por género: 28041
Usuario con más horas jugadas para Action: shinomegami
Horas jugadas por año para shinomegami: 
   year  playtime_forever
0  2012            137029
1  2014              8392
2  2015            530882
{'Usuario con más horas jugadas para Action: shinomegami': 'shinomegami', 'Horas jugadas': [{'Año': 2012, 'Horas': 137029}, {'Año': 2014, 'Horas': 8392}, {'Año': 2015, 'Horas': 530882}]}
